In [2]:
%config Completer.use_jedi = False

In [3]:
cd NN_files/

/Users/cmougan/Desktop/Novartis2021/NN_files


In [9]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use("seaborn")
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import mean_squared_error

from nnet import ReadDataset, Net
import time
from loss_functions import interval_score_loss



tic = time.time()


In [10]:
from pytorch_tabnet.tab_model import TabNetClassifier,TabNetRegressor

In [11]:
import random
from eda.checker import check_train_test

random.seed(0)

sales_train = pd.read_csv("../data/data_raw/sales_train.csv")
df_full = pd.read_csv("../data/split.csv")
df_region = pd.read_csv("../data/data_raw/regions.csv")
regions_hcps = pd.read_csv("../data/data_raw/regions_hcps.csv")
activity_features = pd.read_csv("../data/features/activity_features.csv")
brands_3_12 = pd.read_csv("../data/features/brand_3_12_market_features_lagged.csv")
rte_basic = pd.read_csv("../data/features/rte_basic_features.csv").drop(
    columns=["sales", "validation"]
)

market_size = pd.read_csv("../data/market_size.csv")

# For reproducibility
random.seed(0)
VAL_SIZE = 38
SUBMISSION_NAME = "empty_extractor_target_encoder"
RETRAIN = True

# %% Training weights
market_size = (
    market_size
    .assign(weight=lambda x: 100 / x['sales'])
    .rename(columns={"sales": 'market_size'})
)

market_size

# %% Add region data
df_feats = df_full.merge(df_region, on="region", how="left")
df_feats = pd.merge(left=df_feats, right=regions_hcps, how="left", on="region")
df_feats = df_feats.merge(
    activity_features, on=["month", "region", "brand"], how="left"
)
df_feats = df_feats.merge(rte_basic, on=["month", "region", "brand"], how="left")
df_feats = df_feats.merge(brands_3_12, on=["month", "region"], how="left")
df_feats["whichBrand"] = np.where(df_feats.brand == "brand_1", 1, 0)

df_feats = df_feats.merge(market_size, on='region', how="left")

df_feats['month_brand'] = df_feats.month + '_' + df_feats.brand

# drop sum variables
cols_to_drop = ["region", "sales", "validation", "market_size", "weight"]

# %% Split train val test
X_train = df_feats.query("validation == 0").drop(columns=cols_to_drop)
y_train = df_feats.query("validation == 0").sales
weights_train = df_feats.query("validation == 0").weight

X_val = df_feats.query("validation == 1").drop(columns=cols_to_drop)
y_val = df_feats.query("validation == 1").sales

X_full = df_feats.query("validation.notnull()", engine="python").drop(
    columns=cols_to_drop
)
y_full = df_feats.query("validation.notnull()", engine="python").sales
weights_full = df_feats.query("validation.notnull()", engine="python").weight

X_test = df_feats.query("validation.isnull()", engine="python").drop(
    columns=cols_to_drop
)
y_test = df_feats.query("validation.isnull()", engine="python").sales



In [13]:
clf = TabNetRegressor()

Device used : cpu


,population,area_x,pci16_x,pci18_x,area_y,pci16_y,pci18_y,Internal medicine,Internal medicine / pneumology,General practicioner,...,sales_brand_12_marketshift6,sales_brand_12_marketshift7,sales_brand_12_marketshift8,sales_brand_12_marketshift9,sales_brand_12_marketshift10,sales_brand_12_marketshift11,sales_brand_3_market_per_region,sales_brand_12_market_per_region,whichBrand,month_brand
0,381958,1372.440545,38799.972381,40214.895762,1372.440545,38799.972381,40214.895762,315,18,264,...,480402.71,162082.27,328934.96,267012.51,388414.99,165273.82,18755013.60,10666956.92,1,2020-07_brand_1
1,381958,1372.440545,38799.972381,40214.895762,1372.440545,38799.972381,40214.895762,315,18,264,...,480402.71,162082.27,328934.96,267012.51,388414.99,165273.82,18755013.60,10666956.92,0,2020-07_brand_2
4,318026,2574.943849,42578.018793,44456.389595,2574.943849,42578.018793,44456.389595,153,5,182,...,444736.78,320960.48,234047.10,308306.64,272737.97,580401.41,9541553.05,4650556.48,1,2020-07_brand_1
5,318026,2574.943849,42578.018793,44456.389595,2574.943849,42578.018793,44456.389595,153,5,182,...,444736.78,320960.48,234047.10,308306.64,272737.97,580401.41,9541553.05,4650556.48,0,2020-07_brand_2
6,343861,3639.373478,46856.492170,48564.228841,3639.373478,46856.492170,48564.228841,181,11,206,...,437493.59,476524.07,291943.55,287114.79,301746.46,254380.79,8111375.34,4712618.02,1,2020-07_brand_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4223,317306,2356.435878,45486.027298,47191.593233,2356.435878,45486.027298,47191.593233,316,14,283,...,573747.72,108904.07,320503.96,283027.29,150847.10,196143.04,11343941.68,6432939.58,0,2021-08_brand_2
4224,351807,495.716787,41728.041629,42871.901034,495.716787,41728.041629,42871.901034,221,16,175,...,179091.65,573747.72,108904.07,320503.96,283027.29,150847.10,14411319.55,7862356.96,1,2021-08_brand_1
4225,351807,495.716787,41728.041629,42871.901034,495.716787,41728.041629,42871.901034,221,16,175,...,179091.65,573747.72,108904.07,320503.96,283027.29,150847.10,14411319.55,7862356.96,0,2021-08_brand_2
4226,536929,3415.126006,41281.758615,41579.093639,3415.126006,41281.758615,41579.093639,444,16,303,...,366187.05,213530.17,179091.65,573747.72,108904.07,320503.96,19401482.33,9670167.81,1,2021-08_brand_1


In [17]:

clf.fit(X_train.drop(columns=['month','brand']),y_train)

ValueError: could not convert string to float: '2020-07_brand_1'